In [ ]:
import init


from analisys.bkt_analisys import *
from analisys.data_analisys import *
from analisys.graph_analisys import *
from analisys.optmizer_analisys import *

import time
import timeit
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_score, accuracy_score, precision_recall_curve, auc, make_scorer, recall_score
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import os
os.listdir()

In [ ]:
start_data = '2021.01.01' #tem que colocar um dia antes no start_date
end_data = '2021.06.31'   #tem que colocar um dia depois no end_date
timeframe = 60
symbol = 'AZUL4'


# trinamento
start_train = "2021.01.01"
end_train = "2021.03.31"

# teste
start_test = "2021.04.01"
end_test = "2021.05.01"

# Construcao dos alvos - Expandir Testes Posteriormente de forma automatica com periodos mais longos

periodos = 1


#??
corte = 10


In [ ]:
hist = mt5_singlehistoricaldata(symbol,timeframe,start_data, end_data, raw = True)

df1 = pd.DataFrame(hist)
df1.index = df1['date']

df1.rename(columns={ 'open': 'Abertura','high': 'Maxima','low': 'Minima','close': 'Fechamento' },inplace=True)
df1['Data'] = df1.index

df1 = df1.set_index("Data")

# lag de X periodos
df1["Lag1"] = df1["Fechamento"].shift(-periodos)
# Alvo 1 - Retorno
df1["Retorno"] = df1["Fechamento"].pct_change(periodos)
df1["Alvo1"] = df1["Retorno"].shift(-periodos)
# Variaçao em Pontos do alvo
df1["Pontos"] = (df1["Fechamento"] - df1["Fechamento"].shift(1))
df1["Pontos"] = df1["Pontos"].shift(-periodos)

df1["Alvo_Bin"] = np.where(df1['Alvo1'] > df1["Retorno"].describe()[6]/2 , 1, np.where(df1['Alvo1'] < df1["Retorno"].describe()[4]/2, -1, 0))

In [ ]:
# Construçao das Variaveis
# Desvio Padrao de 5 dias
df1["std5"] = df1["Retorno"].rolling(5).std()
# Desvio Padrao de 10 dias
df1["std10"] = df1["Retorno"].rolling(10).std()
# Desvio Padrao de 15 dias
df1["std15"] = df1["Retorno"].rolling(15).std()
# Proporçao do corpo do candle em relacao ao range do dia
df1["prop"] = (df1["Fechamento"]-df1["Abertura"])/(df1["Maxima"]-df1["Minima"])
# Direçao do dia atual
df1["dir_D"] = np.where(df1['Fechamento'] > df1['Abertura'] , 1, 0)
# Direçao D-1
df1["dir_D-1"] = df1["dir_D"].shift(1)
# Direçao D-2
df1["dir_D-2"] = df1["dir_D"].shift(2)
# Direçao D-3
df1["dir_D-3"] = df1["dir_D"].shift(3)
# Media Movel de 15 dias std5
df1["mm_std5"] = df1["std5"].rolling(15).mean()
# Media Movel de 15 dias std5
df1["mm_std10"] = df1["std10"].rolling(15).mean()
# Media Movel de 15 dias std5
df1["mm_std15"] = df1["std15"].rolling(15).mean()
# RSL std5
df1["RSL_std5"] = (df1["std5"]/df1["std5"].rolling(15).mean())-1
# RSL std10
df1["RSL_std10"] = (df1["std10"]/df1["std10"].rolling(15).mean())-1
# RSL std15
df1["RSL_std15"] = (df1["std15"]/df1["std15"].rolling(15).mean())-1
# Aplicando a fisica
p = 5 
# Velocdade de p dias
df1["v"] = (df1["Retorno"] - df1["Retorno"].shift(p))/p
# Aceleraçao de p dias
df1["a"] = (df1["v"] - df1["v"].shift(p))/p
# Força
# Calculando a massa
df1["m"] = df1["Retorno"].rolling(p).sum()
df1["f"] = df1["m"]*df1["a"]
# Energia cinética
df1["T"] = 0.5*df1["m"]*df1["v"]*df1["v"]

# Trabalho 
# cateto_oposto
cat_op = df1["Retorno"].rolling(p).sum()-df1["Retorno"].rolling(1).sum()
cat_ad = p
df1["cat"] = cat_op/cat_ad
df1 = df1.dropna(axis = 0)
df1["w"] = df1["f"]*np.cos(np.arctan(df1["cat"]))
# Energia potencial
df1["k"] = cat_op*df1["m"]
# Torque
df1["tau"] = df1["f"]*np.sin(np.arctan(df1["cat"]))
# Momentum
df1["M"] = df1["m"]*df1["v"]
# Gravidade
df1["g"] = df1["m"]*df1["Retorno"].rolling(p).sum()/(df1["m"]/df1["Retorno"].rolling(p).sum())*(df1["m"]/df1["Retorno"].rolling(p).sum())


df1_filtrado = df1.dropna(axis = 0) 
df1_filtrado= df1_filtrado.drop(["std5","std10","std15","mm_std5","mm_std10","mm_std15"], axis = 1)


df1_filtrado["RSL_std5"] = pd.qcut(df1_filtrado["RSL_std5"], corte, labels = False)
df1_filtrado["RSL_std10"] = pd.qcut(df1_filtrado["RSL_std10"], corte, labels = False)
df1_filtrado["RSL_std15"] = pd.qcut(df1_filtrado["RSL_std15"], corte, labels = False)
df1_filtrado["v"] = pd.qcut(df1_filtrado["v"], corte, labels = False)
df1_filtrado["a"] = pd.qcut(df1_filtrado["a"], corte, labels = False)
df1_filtrado["m"] = pd.qcut(df1_filtrado["m"], corte, labels = False)
df1_filtrado["f"] = pd.qcut(df1_filtrado["f"], corte, labels = False)
df1_filtrado["T"] = pd.qcut(df1_filtrado["T"], corte, labels = False)
df1_filtrado["w"] = pd.qcut(df1_filtrado["w"], corte, labels = False)
df1_filtrado["k"] = pd.qcut(df1_filtrado["k"], corte, labels = False)
df1_filtrado["tau"] = pd.qcut(df1_filtrado["tau"], corte, labels = False)
df1_filtrado["M"] = pd.qcut(df1_filtrado["M"], corte, labels = False)
df1_filtrado["g"] = pd.qcut(df1_filtrado["g"], corte, labels = False)

In [ ]:
plt.figure(figsize = (18,5))
plt.hist(df1["Retorno"], bins = 25);

plt.legend()
plt.title("Distribuiçao Retornos");